In [1]:
from feast import FeatureStore
from pyspark.sql import SparkSession
from feast.stream_processor import SparkProcessorConfig
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell"
spark = SparkSession.builder.master("local").appName("feast-spark").getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", 5)

Ivy Default Cache set to: /Users/kevinzhang/.ivy2/cache
The jars for the packages stored in: /Users/kevinzhang/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f54d3202-3c9b-4d85-8bb8-8806c804744a;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/kevinzhang/.pyenv/versions/3.8.12/envs/feast_env/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in local-m2-cache
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 224ms :: artifacts dl 8ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.4-3 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in [default]
	org.apache.kafka#kafka-clients;2.4.1 from central in [default]
	org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 from central in [default]
	org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 from central in [default]
	org.lz4#lz4-java;1.7.1 from central in [default]
	org.

### Fetch training data from offline store

In [8]:
!feast apply


/Users/kevinzhang/tecton-ai/offline_store/feast/sdk/python/feast/feature_view.py:178: DeprecationWarning: The `entities` parameter should be a list of `Entity` objects. Feast 0.23 and onwards will not support passing in a list of strings to define entities.
  warnings.warn(
/Users/kevinzhang/tecton-ai/offline_store/feast/sdk/python/feast/stream_feature_view.py:61: RuntimeWarning: Stream Feature Views are experimental features in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(
Updated stream feature view driver_hourly_stats
	aggregations: [column: "miles_driven"
function: "sum"
time_window {
  seconds: 86400
}
slide_interval {
  seconds: 18000
}
, column: "miles_driven"
function: "count"
time_window {
  seconds: 86400
}
slide_interval {
  seconds: 18000
}
, column: "miles_driven"
function: "count"
time_window {
  seconds: 86400
}
slide_interval {
  seconds: 18000
}
] -> [column: "miles_driven"
function: "sum"
time_w

In [7]:
!feast materialize-incremental $(date +%Y-%m-%d)


Materializing 1 feature views to 2022-06-08 17:00:00-07:00 into the sqlite online store.

normal_stats from 2022-06-07 17:00:00-07:00 to 2022-06-08 17:00:00-07:00:
0it [00:00, ?it/s]


#### The normal_stats feature view contains the data without streaming directly from the file parquet file. We fetch some feature data here from the normal_stats feature view. 

In [9]:
fs = FeatureStore(".")

features = fs.get_online_features(
    features=[
        "normal_stats:conv_rate",
        "normal_stats:acc_rate",
        "normal_stats:miles_driven"
    ],
    entity_rows=[
        {
            "driver_id": 1001,
        }
    ],
).to_dict(include_event_timestamps=True)

def print_online_features(features):
    for key, value in sorted(features.items()):
        print(key, " : ", value)

print_online_features(features)

acc_rate  :  [0.4075707495212555]
acc_rate__ts  :  [1647266400]
conv_rate  :  [0.4045884609222412]
conv_rate__ts  :  [1647266400]
driver_id  :  [1001]
driver_id__ts  :  [0]
miles_driven  :  [46.53329086303711]
miles_driven__ts  :  [1647266400]


In [10]:
sfv = fs.list_stream_feature_views()[0]
sfv

<StreamFeatureView(aggregations = [<feast.aggregation.Aggregation object at 0x17284a250>, <feast.aggregation.Aggregation object at 0x17284a6d0>, <feast.aggregation.Aggregation object at 0x17284a7c0>], mode = spark, timestamp_field = event_timestamp, udf = None, name = driver_hourly_stats, entities = ['driver'], stream_source = {
  "type": "STREAM_KAFKA",
  "timestampField": "event_timestamp",
  "kafkaOptions": {
    "bootstrapServers": "localhost:9092",
    "topic": "drivers",
    "messageFormat": {
      "jsonFormat": {
        "schemaJson": "driver_id integer, miles_driven double, event_timestamp timestamp, conv_rate double, acc_rate double"
      }
    },
    "watermark": "300s"
  },
  "name": "driver_stats_stream",
  "batchSource": {
    "type": "BATCH_FILE",
    "timestampField": "event_timestamp",
    "createdTimestampColumn": "created",
    "fileOptions": {
      "uri": "data/driver_stats.parquet"
    },
    "name": "data/driver_stats.parquet"
  }
}, batch_source = {
  "type": "

#### Set up the configuration of your spark node/cluster and pass in a SparkSession object. 
- Pass in the processor config and ingest stream feature view data from the kafka source data. 
- Since no udfs are used, the dataframe that is created will be modified and written to the online store. 

In [6]:
# should only return miles_driven since there is no fresher data for conv_rate and acc_rate
config = SparkProcessorConfig(mode="spark", source="kafka", spark_session=spark)
query = fs.ingest_stream_feature_view(sfv_name=sfv.name, processor_config=config)


22/06/07 13:43:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
/Users/kevinzhang/tecton-ai/offline_store/feast/sdk/python/feast/stream_feature_view.py:61: RuntimeWarning: Stream Feature Views are experimental features in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/kevinzhang/.pyenv/versions/3.8.12/envs/feast_env/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/kevinzhang/.pyenv/versions/3.8.12/envs/feast_env/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/kevinzhang/.pyenv/versions/3.8.12/lib/python3.8/socket.py", line 669, in readinto
    retu

KeyboardInterrupt: 

In [11]:
# should only return miles_driven since there is no fresher data for conv_rate and acc_rate
config = SparkProcessorConfig(mode="spark", source="kafka", spark_session=spark)
df = fs.transform_stream_data(sfv_name=sfv.name, processor_config=config)
query = df.writeStream \
    .queryName("aggregates") \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()



22/06/09 10:56:16 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/35/ftzv2s4d2nb0fbr0gxfr2frw0000gn/T/temporary-b9599b78-be7c-44ce-8820-e133aaa67e78. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/09 10:56:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+---------+-------------------+---------------------+-------------------+
|window|driver_id|miles_driven_sum_0s|miles_driven_count_0s|miles_driven_max_0s|
+------+---------+-------------------+---------------------+-------------------+
+------+---------+-------------------+---------------------+-------------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+------+---------+-------------------+---------------------+-------------------+
|window|driver_id|miles_driven_sum_0s|miles_driven_count_0s|miles_driven_max_0s|
+------+---------+-------------------+---------------------+-------------------+
+------+---------+-------------------+---------------------+-------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+------+---------+-------------------+---------------------

-------------------------------------------
Batch: 19
-------------------------------------------
+------+---------+-------------------+---------------------+-------------------+
|window|driver_id|miles_driven_sum_0s|miles_driven_count_0s|miles_driven_max_0s|
+------+---------+-------------------+---------------------+-------------------+
+------+---------+-------------------+---------------------+-------------------+

-------------------------------------------
Batch: 20
-------------------------------------------
+------+---------+-------------------+---------------------+-------------------+
|window|driver_id|miles_driven_sum_0s|miles_driven_count_0s|miles_driven_max_0s|
+------+---------+-------------------+---------------------+-------------------+
+------+---------+-------------------+---------------------+-------------------+

-------------------------------------------
Batch: 21
-------------------------------------------
+------+---------+-------------------+------------------

-------------------------------------------
Batch: 36
-------------------------------------------
+------+---------+-------------------+---------------------+-------------------+
|window|driver_id|miles_driven_sum_0s|miles_driven_count_0s|miles_driven_max_0s|
+------+---------+-------------------+---------------------+-------------------+
+------+---------+-------------------+---------------------+-------------------+

-------------------------------------------
Batch: 37
-------------------------------------------
+------+---------+-------------------+---------------------+-------------------+
|window|driver_id|miles_driven_sum_0s|miles_driven_count_0s|miles_driven_max_0s|
+------+---------+-------------------+---------------------+-------------------+
+------+---------+-------------------+---------------------+-------------------+

-------------------------------------------
Batch: 38
-------------------------------------------
+------+---------+-------------------+------------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/kevinzhang/.pyenv/versions/3.8.12/envs/feast_env/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/kevinzhang/.pyenv/versions/3.8.12/envs/feast_env/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/kevinzhang/.pyenv/versions/3.8.12/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


-------------------------------------------
Batch: 42
-------------------------------------------
+------+---------+-------------------+---------------------+-------------------+
|window|driver_id|miles_driven_sum_0s|miles_driven_count_0s|miles_driven_max_0s|
+------+---------+-------------------+---------------------+-------------------+
+------+---------+-------------------+---------------------+-------------------+



KeyboardInterrupt: 

In [12]:
query.stop()

22/06/09 10:56:55 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@28f65d1 is aborting.
22/06/09 10:56:55 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@28f65d1 aborted.
22/06/09 10:56:56 ERROR Utils: Aborting task
org.apache.spark.executor.CommitDeniedException: Commit denied for partition 4 (task 641, attempt 0, stage 165.0)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.commitDeniedError(QueryExecutionErrors.scala:620)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.$anonfun$run$1(WriteToDataSourceV2Exec.scala:433)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1496)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.run(WriteToDataSourceV2Exec.scala:457)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.$anonfun$writeWithV2$2(WriteTo

### Retrieve features from the driver_hourly_stats stream feature view.
- You should see different miles_driven features than the ones you saw in normal stats since the data being pulled is edited to have a fresher(newer) creation timestamp.

In [7]:
# streaming input has rows of miles_driven features that are more fresh than the ones in the batch source
features = fs.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:miles_driven"
    ],
    entity_rows=[
        {
            "driver_id": 1001,
        }
    ],
).to_dict(include_event_timestamps=True)

print_online_features(features)

acc_rate  :  [None]
acc_rate__ts  :  [1678006800]
conv_rate  :  [None]
conv_rate__ts  :  [1678006800]
driver_id  :  [1001]
driver_id__ts  :  [0]
miles_driven  :  [45.377933502197266]
miles_driven__ts  :  [1678006800]


/Users/kevinzhang/tecton-ai/offline_store/feast/sdk/python/feast/stream_feature_view.py:61: RuntimeWarning: Stream Feature Views are experimental features in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(
/Users/kevinzhang/tecton-ai/offline_store/feast/sdk/python/feast/stream_feature_view.py:61: RuntimeWarning: Stream Feature Views are experimental features in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(
